In [25]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import pandas as pd

In [26]:
# Launch the Chrome driver
driver = webdriver.Chrome('C:\chromedriver\chromedriver')

# Login to LinkedIn
driver.get("https://linkedin.com/uas/login")
username = driver.find_element(By.ID, "username")
username.send_keys("your_email_address") # Enter your email address
pword = driver.find_element(By.ID,"password")
pword.send_keys("your_password") # Enter your password
# Click on the submit button to login
driver.find_element(By.XPATH,"//button[@type='submit']").click()

C:\Users\ADS\AppData\Local\Temp\ipykernel_18404\3391605287.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:\chromedriver\chromedriver')


In [27]:
driver.get("https://www.linkedin.com/school/institut-national-de-statistique-et-d-economie-appliquee-insea/people/")
#choose year between 1999 and 2018
year = driver.find_element(By.ID,"people-search-year-end")
year.send_keys("2018") 
driver.find_element(By.XPATH,"//input[@class='org-people__input-year fl']").click()

In [28]:
# Scroll the page to load more profiles
rep = 50 #determine the rep enough to scroll all the page
last_height = driver.execute_script("return document.body.scrollHeight")

for i in range(rep):
  driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
  time.sleep(5)
  new_height = driver.execute_script("return document.body.scrollHeight")
  if new_height == last_height:
    break
  new_height = last_height

In [29]:
# Extract HTML source code of the page and parse it using BeautifulSoup
src = driver.page_source 
soup = BeautifulSoup(src, 'lxml')
# Find the target div element containing all the profiles
pav = soup.find('div', {'class' : 'org-grid__content-height-enforcer'})
all_links = pav.find_all('a', {'class' : 'app-aware-link link-without-visited-state'})

In [30]:
# Extract profile links for all the profiles on the page
profilesID = []
for link in all_links:
  profilesID.append(link.get('href'))

In [31]:
# Create empty lists to store the extracted data
names = []
companies = []
locs = []
works= []

# Iterate over all profile links and extract the desired data
for profileID in profilesID[1:20]: # consider only 20 profiles for this example
    fulllink = profileID
    driver.get(fulllink)
    time.sleep(8)
    src = driver.page_source
    soup = BeautifulSoup(src, 'lxml')
    
    # Extract name
    intro = soup.find('div', {'class': 'mt2 relative'})
    name_loc = intro.find("h1")
    name = name_loc.get_text().strip()
    names.append(name) 
    
    # Extract location
    location_loc = intro.find("span", {'class': 'text-body-small inline t-black--light break-words'})
    location = location_loc.get_text().strip()
    locs.append(location)
    
    # Extract company
    job = intro.find_all("span", {'class': 'text-body-small'})
    company = job[1].get_text().strip()
    companies.append(company)

    # Extract current job title
    works_at_loc = intro.find("div", {'class': 'text-body-medium'})
    works_at = works_at_loc.get_text().strip()
    works.append(works_at)

In [32]:
df = pd.DataFrame(list(zip(names, companies, locs, works)), 
               columns =['names', 'companies', 'locations', 'works']) 

In [34]:
df = df.drop('names', axis=1)

In [35]:
df.to_csv('INSEA_Alumni.csv', index=False)